In [46]:

%load_ext autoreload
%autoreload 2
from loaders.loaders import *
from dataset_dict import dataset_dict, output_path
from loaders.RecordDistributor import RecordDistributor
from loaders.CleanedEnglishDataset import WikiBookEn
from loaders.OtherDatasets import *
from loaders.PilePythonDataset import PilePythonDataset

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [57]:
from datasets import load_dataset
#ds= load_dataset("suolyer/pile_stackexchange",streaming=True,split="validation")

#ds=PythonCodeDataset()
#ds=OpenMathInstructJa()
ds=PilePythonDataset("../data/original_dump/python/")
loader=iter(ds)

python0000.jsonl.zst already exists.
python0001.jsonl.zst already exists.
python0002.jsonl.zst already exists.
python0003.jsonl.zst already exists.
python0004.jsonl.zst already exists.
python0005.jsonl.zst already exists.
python0006.jsonl.zst already exists.
python0007.jsonl.zst already exists.
python0008.jsonl.zst already exists.
python0009.jsonl.zst already exists.
python0010.jsonl.zst already exists.
python0011.jsonl.zst already exists.
python0012.jsonl.zst already exists.
python0013.jsonl.zst already exists.
python0014.jsonl.zst already exists.
python0015.jsonl.zst already exists.
python0016.jsonl.zst already exists.
python0017.jsonl.zst already exists.
python0018.jsonl.zst already exists.
python0019.jsonl.zst already exists.
python0020.jsonl.zst already exists.
python0021.jsonl.zst already exists.
python0022.jsonl.zst already exists.
python0023.jsonl.zst already exists.
python0024.jsonl.zst already exists.
python0025.jsonl.zst already exists.
python0026.jsonl.zst already exists.
p

In [58]:
ds.current_loader

<generator object Dataset.__iter__ at 0x14a7041bd5a0>

In [63]:
from tqdm import tqdm
for i in tqdm(range(10**4)):
    next(loader)

  0%|          | 0/10000 [00:00<?, ?it/s]

100%|██████████| 10000/10000 [00:01<00:00, 7297.39it/s]


In [64]:
print(next(loader)["text"])

import numpy as np

# Import the model from the auxillary folder
import sys
sys.path.append("aux")
from Example_3_4_aux import myModel

np.random.seed(1)

# Constants:
N = 10
Ntest = 100

H = 15
varType = "float64"

# Analytical solution
real = lambda x,y: np.exp(-x)*(x+y**3)

# Create the training data
X,Y = np.meshgrid(np.linspace(0,1,N),np.linspace(0,1,N))
X = X.flatten(); Y = Y.flatten()
inputs = np.vstack([X,Y])

inputs = np.array(inputs,dtype=varType).T

# Train the class
outputs = np.zeros((inputs.shape[0],1),dtype=varType)

model = myModel(H)
model.trainBfgs(inputs,outputs,maxIter=2000,tol=1e-14)

errUTrain = np.abs(model.call(inputs)-np.expand_dims(real(X,Y),1))

#: Print out the error statistics for the test set
Xtest, Ytest = np.meshgrid(np.linspace(0,1,Ntest),np.linspace(0,1,Ntest))
Xtest = Xtest.flatten(); Ytest = Ytest.flatten()

test = np.vstack([Xtest,Ytest]).T
test = np.array(test,dtype=varType)
u = model.call(test)
errU = np.abs(u-np.expand_dims(real(Xtest,Ytest),1))
